# Create models

Create a word2vec model using Gensim library. 

In [1]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

In [2]:
import dhlab as dh
import pandas as pd
from dhlab.nbtokenizer import tokenize
dh.css()

## Model production

The texts are in a file, one line pr. text.

In [16]:
sentence_file = 'nou1997all.txt'

In [17]:
%%time
sentences = LineSentence(sentence_file)
model = Word2Vec(sentences= sentences, vector_size=100, window=5, min_count=5, workers=10)

CPU times: user 1.67 s, sys: 30.2 ms, total: 1.7 s
Wall time: 657 ms


In [18]:
model.save(f"{sentence_file.split('.')[0]}.model")